In [30]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, SimpleRNN, LSTM, GRU, Dense

In [31]:
num_words= 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)

In [32]:
maxlen = 100
x_train = pad_sequences(x_train, maxlen=maxlen, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=maxlen, padding='post', truncating='post')

In [33]:
embedding =100
model = Sequential([
    Embedding(input_dim=num_words, output_dim=embedding, input_length=maxlen),
    Bidirectional(LSTM(5)),
    Dense(1, activation='sigmoid')
])
model.build()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train,
                    batch_size=num_words,
                    epochs=1)
model.summary()


c:\Users\HP\Desktop\VSCode\Bidirection_lstm\venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.5221 - loss: 0.6925


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 100, 100)       │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 10)             │         4,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,012,755 (11.49 MB)

 Trainable params: 1,004,251 (3.83 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,008,504 (7.66 MB)

In [34]:
train_loss, train_acc = model.evaluate(x_test, y_test, verbose=2)

782/782 - 3s - 4ms/step - accuracy: 0.5671 - loss: 0.6906


In [35]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [36]:
log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [37]:
log_dir

'logs/fit20240724-130616'

In [38]:
model_callbacks = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [39]:
import mlflow

In [42]:
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, callbacks=[model_callbacks, earLy_stop]
)

Epoch 1/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 19s 24ms/step - accuracy: 0.8500 - loss: 0.3576 - val_accuracy: 0.8118 - val_loss: 0.4315
Epoch 2/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 18s 23ms/step - accuracy: 0.9039 - loss: 0.2474 - val_accuracy: 0.8047 - val_loss: 0.4704
Epoch 3/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 18s 23ms/step - accuracy: 0.9456 - loss: 0.1599 - val_accuracy: 0.7937 - val_loss: 0.5306
Epoch 4/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - accuracy: 0.9636 - loss: 0.1126 - val_accuracy: 0.8006 - val_loss: 0.6226
Epoch 5/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 23s 29ms/step - accuracy: 0.9747 - loss: 0.0818 - val_accuracy: 0.7951 - val_loss: 0.6721
Epoch 6/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - accuracy: 0.9876 - loss: 0.0489 - val_accuracy: 0.7854 - val_loss: 0.7176
Epoch 7/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 19s 24ms/step - accuracy: 0.9911 - loss: 0.0345 - val_accuracy: 0.7934 - val_loss: 0.8523
Epoch 8/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 19s 25ms/step - accuracy: 0.9962 - loss: 0

In [41]:
earLy_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [43]:
from mlflow.models.signature import infer_signature
model_sign = infer_signature(x_test, model.predict(x_test))

782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step


In [44]:
exp_timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
exp_name= "IMDB_bidirection_lstm_time:" + exp_timestamp
#exp_name = "IMDB_bidirection_lstm"
run_timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
run_name = "IMDB_bidirection_lstm_run:" + run_timestamp

In [46]:
def tf_experiment_tracking(exp_name, run_name, train_loss, train_acc, model_sign):
    mlflow.set_experiment(exp_name)
    with mlflow.start_run(run_name=run_name):
        mlflow.log_param("batch_size", num_words )
        mlflow.log_param("learning_rate", 0.001)
        mlflow.log_param("epochs", 100)
        mlflow.log_metric("train_loss", train_loss)
        mlflow.log_metric("train_accuracy", train_acc)
        mlflow.tensorflow.log_model(model, "IMDB", signature=model_sign)
    mlflow.end_run()

In [47]:
tf_experiment_tracking(exp_name, run_name, train_loss, train_acc, model_sign)

2024/07/24 13:10:33 INFO mlflow.tracking.fluent: Experiment with name 'IMDB_bidirection_lstm_time:20240724-131022' does not exist. Creating a new experiment.


In [53]:
from tensorflow.keras.models import load_model
lm = load_model('models\BI_LSTM_model.h5')

<>:2: SyntaxWarning: invalid escape sequence '\B'
<>:2: SyntaxWarning: invalid escape sequence '\B'
C:\Users\HP\AppData\Local\Temp\ipykernel_13740\59042200.py:2: SyntaxWarning: invalid escape sequence '\B'
  lm = load_model('models\BI_LSTM_model.h5')


ValueError: Layer count mismatch when loading weights from file. Model expected 1 layers, found 3 saved layers.

In [25]:
from pathlib import Path
from service.bentoml_load_model import load_model_and_save_it_in_bentoml


In [29]:
import bentoml
bentoml.tensorflow.save_model( model,
    "models/saved_model",
    signatures={"batchable": True})

TypeError: argument of type 'Sequential' is not iterable